In [1]:
import requests
import json
import time
import random
import os
import pandas as pd

In [2]:
from dotenv import load_dotenv
from datetime import datetime, timedelta
from pytz import timezone

load_dotenv()

BEARERTOKEN = os.getenv('BEARERTOKEN')

Source Code : https://medium.com/data-analytics-at-nesta/all-you-need-to-get-started-with-twitter-api-v2-using-python-6cd4be4d90fe

In [3]:
# Endpoint URL
endpoint_url = "https://api.twitter.com/2/tweets/search/recent"

#Timed settings with ISO 8601 format
local_time = timezone("Asia/Jakarta").localize(datetime(2025, 1, 8, 11, 0, 0))
start_time = local_time.astimezone(timezone("UTC")).strftime("%Y-%m-%dT%H:%M:%SZ")

In [4]:
# Query parameters
#query_parameters = {
#    "query": '("makan bergizi gratis" OR "makan siang gratis" OR "MBG") lang:id -is:retweet',
#    "tweet.fields": "id,text,author_id,created_at",
#    "start_time": start_time,
#    "max_results": 10,
#}

In [5]:
def request_headers(bearer_token: str) -> dict:
    """
    Sets up the request headers. 
    Returns a dictionary summarising the bearer token authentication details.
    """
    return {"Authorization": "Bearer {}".format(bearer_token)}

headers = request_headers(BEARERTOKEN)

In [6]:
def connect_to_endpoint(endpoint_url: str, headers: dict, parameters: dict) -> json:
    """
    Connects to the endpoint and requests data.
    Returns a json with Twitter data if a 200 status code is yielded.
    Programme stops if there is a problem with the request and sleeps
    if there is a temporary problem accessing the endpoint.
    """
    response = requests.request(
        "GET", url=endpoint_url, headers=headers, params=parameters
    )
    response_status_code = response.status_code
    if response_status_code != 200:
        if response_status_code >= 400 and response_status_code < 500:
            raise Exception(
                "Cannot get data, the program will stop!\nHTTP {}: {}".format(
                    response_status_code, response.text
                )
            )
        
        sleep_seconds = random.randint(5, 60)
        print(
            "Cannot get data, your program will sleep for {} seconds...\nHTTP {}: {}".format(
                sleep_seconds, response_status_code, response.text
            )
        )
        time.sleep(sleep_seconds)
        return connect_to_endpoint(endpoint_url, headers, parameters)
    return response.json()

#uncomment to execute
#json_response = connect_to_endpoint(endpoint_url, headers, query_parameters)

In [23]:
#set up rules
#rules = [
#    {"value": '("makan bergizi gratis" OR "makan siang gratis" OR "MBG") lang:id -is:retweet', "tag": "mbg"},
#]

In [7]:
def process_twitter_data(
    json_response: json,
    query_tag: str,
    tweets_data: pd.DataFrame,
    users_data: pd.DataFrame,
) -> (pd.DataFrame, pd.DataFrame):
    """
    Adds new tweet/user information to the table of
    tweets/users and saves dataframes as pickle files,
    if data is avaiable.
    
    Returns the tweets and users updated dataframes.
    """
    if "data" in json_response.keys():
        new = pd.DataFrame(json_response["data"])
        tweets_data = pd.concat([tweets_data, new])
        tweets_data.to_pickle("tweets_" + query_tag + ".pkl")

        if "users" in json_response["includes"].keys():
            new = pd.DataFrame(json_response["includes"]["users"])
            users_data = pd.concat([users_data, new])
            users_data.drop_duplicates("id", inplace=True)
            users_data.to_pickle("users_" + query_tag + ".pkl")

    return tweets_data, users_data

In [8]:
query_parameters = {
    "tweet.fields": "id,text,author_id,created_at",
    "user.fields": "id,name,username,created_at,description,location,verified",
    "start_time": start_time,
    "max_results": 99,
}

In [9]:
rules = [
    {"value": '("makan bergizi gratis" OR "makan siang gratis") lang:id -is:retweet', "tag": "mbg"},
]

In [10]:
tweets_data = pd.DataFrame()
users_data = pd.DataFrame()

for i in range(len(rules)):
    query_parameters["query"] = rules[i]["value"]
    query_tag = rules[i]["tag"]

    json_response = connect_to_endpoint(endpoint_url, headers, query_parameters)
    tweets_data, users_data = process_twitter_data(
        json_response, query_tag, tweets_data, users_data
    )

    time.sleep(20)

    while "next_token" in json_response["meta"]:
        query_parameters["next_token"] = json_response["meta"]["next_token"]

        json_response = connect_to_endpoint(endpoint_url, headers, query_parameters)
        tweets_data, users_data = process_twitter_data(
            json_response, query_tag, tweets_data, users_data
        )

        time.sleep(20)

Exception: Cannot get data, the program will stop!
HTTP 400: {"errors":[{"parameters":{"start_time":["2025-01-08T04:00Z"]},"message":"Invalid 'start_time':'2025-01-08T04:00Z'. 'start_time' must be on or after 2025-01-11T19:31Z"}],"title":"Invalid Request","detail":"One or more parameters to your request was invalid.","type":"https://api.twitter.com/2/problems/invalid-request"}

In [ ]:
json_response

In [39]:
## ganti nama file

with open('jsonresponse2.json', 'w') as fp: 
    json.dump(json_response, fp)

In [60]:
resultdf = pd.DataFrame(columns=['id', 'edit_history_tweet_ids', 'created_at', 'text', 'author_id'])

for i in range(len(json_response['data'])):
    #print(json_response['data'][i])
    insertdf = pd.DataFrame(json_response['data'][i])
    resultdf = pd.concat([resultdf,insertdf])

In [ ]:
resultdf